In [101]:
%%time
%pdb on

import pandas as pd
import numpy as np
import inro.modeller as _m
import os

# Initialize the Modeller
modeller = _m.Modeller()
import_network_tool = modeller.tool("tmg.input_output.import_network_package")
export_network_tool = modeller.tool('tmg.input_output.export_network_package')


print "starting...."
print 

parameters = {
    "IMPORT_NETWORK_FLAG":True,
    "CLEAR_TRANSIT_FLAG": False,
    "EXPORT_NETWORK_FLAG": True,
    "mode_to_modify": "r",  
    "network_folder": "csfeb_networks",  
    "export_network_folder": "exported_networks",  
    "station_to_node_file": "inputs/station_to_stop_updated.csv", 
    "segment_file": "inputs/segment.csv",
    "header_file": "inputs/header.csv", 
    "capacity_file": "inputs/vehicle.csv", 
    "source_scenarios": [
        {"time_period": "AM", "scenario_id": 10},
        {"time_period": "Midday", "scenario_id": 20},
        {"time_period": "PM", "scenario_id": 30},
        {"time_period": "Evening", "scenario_id": 40},
        {"time_period": "Overnight", "scenario_id": 50},
    ],
    
    "target_scenarios": [
        {"time_period": "AM", "scenario_id": 100},
#         {"time_period": "Midday", "scenario_id": 200},
#         {"time_period": "PM", "scenario_id": 300},
#         {"time_period": "Evening", "scenario_id": 400},
#         {"time_period": "Overnight", "scenario_id": 500},
    ],
}

"""
Parameters Documentation:

    - IMPORT_NETWORK_FLAG (bool): If set to True, network packages will be imported based on 
                            filenames that suggest the time period (e.g., AM, PM). If False, 
                            the databank is assumed to already contain the correct networks, 
                            and no import is necessary.

    - CLEAR_TRANSIT_FLAG (bool): When True, existing transit lines for the specified modes will 
                            be cleared in the target scenarios before any new data is applied. 
                            This is particularly useful for updates or complete overhauls of transit line data.

    - mode_to_modify (str): Identifies the transit mode(s) to be modified, using their respective 
                            codes (e.g., "r" for rail). Multiple modes can be specified, separated 
                            by commas (e.g., "r,b" for both rail and bus).

    - network_folder (str): The directory path containing network package files for import or modification. 
                            Should be left as an empty string ("") if network importation is not part of the process or 
                            set the IMPORT_NETWORK_FLAG to false.

    - station_to_node_file (str): File path to a CSV mapping station IDs to network node IDs. This mapping is crucial 
                            for identifying nodes where stations are located within the network.The CSV file must 
                            contain the following columns: station_name,node_id

    - segment_file (str): File path to a CSV detailing network segments, including attributes such as lengths, 
                            capacities, and other relevant information.The CSV file should contain the following 
                            columns: line_id,from_station,to_station,travel_time,dwell_time,vehicle_id
                            
                            NOTE: station_names in station_to_node_file above must also match the from_station and 
                            to_station station names.

    - header_file (str): File path to a CSV containing header or metadata for the network, potentially including 
                            global configurations or settings applicable to the entire network. The CSV file must 
                            contain the following columns:line_id,description,vehicle_id,am_tph,md_tph,pm_tph,ev_tph


    - capacity_file (str): File path to a CSV with capacity information for various network elements, mostly related to 
                            transit vehicle to inform network simulations or analyses. The CSV must contain the following 
                            columns: consist_type,total_seats,total_standing,total_capacity


    - source_scenarios (list of dicts): A list of dictionaries, each representing a source scenario with keys for 
                            'time_period' (indicating the part of the day) and 'scenario_id' (a unique identifier).
                            e.g: [{"time_period": "AM", "scenario_id": 10}, {"time_period": "Midday", "scenario_id": 20}]

    - target_scenarios (list of dicts): Similar to source_scenarios, but intended for modifications or analyses. 
                            Each dictionary includes 'time_period' and 'scenario_id'. 
                            e.g: [{"time_period": "AM", "scenario_id": 100}, {"time_period": "Midday", "scenario_id": 200}]
"""


def import_network(network_folder, network_file, scenario_id):
    """Imports a network package into a specified scenario."""
    network_package_path = os.path.join(network_folder, network_file)
    scenario_name = network_file[:-4]
    import_network_tool(network_package_path, scenario_id, 'OVERWRITE', True, scenario_name)
    print "Imported network for scenario {}".format(scenario_id)

def load_scenario(scenario_id):
    scenario = modeller.emmebank.scenario(scenario_id)
    if not scenario:
        raise Exception("Scenario {} was not found!".format(scenario_id))
    return scenario
    
def clear_transit_lines(network, modes_to_clear, scenario_id):
    lines_to_delete = [line.id for line in network.transit_lines() if line.mode in modes_to_clear]
    for line_id in lines_to_delete:
        network.delete_transit_line(line_id)
    print "Cleared transit lines for modes \"{}\" in Scenario {}".format(', '.join(str(list(modes_to_clear)[0])), 
                                                                         str(scenario_id))
    
def calculate_shortest_path(from_node, to_node, network, excluded_links,cached_shortest_costs):
    shortest_path_tree = network.shortest_path_tree(
        origin_node_id=from_node, 
        link_costs="length", 
        excluded_links=excluded_links, 
        consider_turns=False, 
        turn_costs=None, 
        max_cost=None
        
    )
    path_nodes = shortest_path_tree.path_to_node(to_node)
    cached_shortest_costs.at[from_node, to_node] = shortest_path_tree.cost_to_node(to_node)
    path_links = shortest_path_tree.path_to_node(to_node)
    path = [link.j_node.number for link in path_links[:-1]] 
    
    return path

def copy_and_publish_scenario(network_to_copy, from_scenario, to_scenario_id):
    """
    Copies a source scenario to a target scenario ID and publishes the network.
    If the target scenario already exists, it is deleted before copying.

    Args:
        network_to_copy: The network to copy.
        from_scenario: The source scenario to copy.
        to_scenario_id: The ID of the target scenario to copy to.

    Returns:
        A message success or failure message.
    """
    
    # Check if the target scenario already exists and delete it if it does
    if modeller.emmebank.scenario(to_scenario_id) is not None:
        print("Deleting existing target scenario: {}".format(to_scenario_id))
        modeller.emmebank.delete_scenario(to_scenario_id)
    
    # Copy the source scenario to the target scenario ID
    if from_scenario is None:
        return "Source scenario {} not found.".format(from_scenario.id)
    
    to_scenario = modeller.emmebank.copy_scenario(from_scenario.number, to_scenario_id)
    print("Created target scenario: {}".format(to_scenario.id))
    
    # Publish the network of the source scenario to the target scenario
    to_scenario.publish_network(network_to_copy)
    
    return "Successfully copied and published network from scenario {} to scenario {}.".format(from_scenario.id, to_scenario_id)


def calculate_attributes(line_id, grouped_segments_by_line_dict, cached_shortest_paths, cached_shortest_costs):
    attrib = {
        "speeds": [],
        "itinerary": [],
        "stop_flags": [],
        "dwell_times": [0.0]
    }

    if line_id in grouped_segments_by_line_dict:
        for segment in grouped_segments_by_line_dict[line_id]:
            from_node = segment["from_node_id"]
            to_node = segment["to_node_id"]
            
            attrib["itinerary"].append(int(from_node))
            path_nodes = cached_shortest_paths[from_node, to_node]
            attrib["itinerary"] += path_nodes

            length = cached_shortest_costs.at[int(from_node), int(to_node)]
            time = segment["travel_time"]
            speed = length / time * 60.0

            for _ in range(len(path_nodes) + 1): 
                attrib["speeds"].append(speed)

            attrib["stop_flags"].append(True)
            for _ in range(len(path_nodes)):
                attrib["stop_flags"].append(False)
                attrib["dwell_times"].append(0.0)
            attrib["dwell_times"].append(segment["dwell_time"])
        
        attrib["itinerary"].append(int(to_node))
    return attrib


def change_vehicle_properties(vehicle, description="", auto_eq=0.0, seated_cap=0, total_cap=0):
    
    if description:
        vehicle.description = description
    
    if auto_eq != 0.0:
        vehicle.auto_equivalent = auto_eq
        
    if seated_cap != 0:
        vehicle.seated_capacity = seated_cap
    
    # Ensure total capacity is not less than seated capacity
    total_cap = max(total_cap, seated_cap)
    
    if total_cap != 0:
        vehicle.total_capacity = total_cap

def check_and_create_vehicles(network, capacity_dict):
    new_vehicle_id_dict = {}
    existing_veh_desc = [str(veh.description) for veh in network.transit_vehicles()]
    existing_veh_id = sorted([veh.number for veh in network.transit_vehicles()])
    
    for vehicle_desc, veh_attrib in capacity_dict.iteritems():
        if vehicle_desc not in existing_veh_desc:
            new_veh_id = max(existing_veh_id) + 1
            new_vehicle = network.create_transit_vehicle(new_veh_id, "r")
            seated_cap = veh_attrib["total_seats"]
            total_cap = veh_attrib["total_capacity"]
            description = vehicle_desc

            change_vehicle_properties(new_vehicle,
                                       description=description,
                                       auto_eq=0.0,
                                       seated_cap=seated_cap,
                                       total_cap=total_cap)
            
            existing_veh_id.append(new_veh_id)
            new_vehicle_id_dict[vehicle_desc] = new_veh_id
    
    return new_vehicle_id_dict

def create_transit_routes(line_id, network, trains_per_hour, itinerary, vehicle_id, description,scenario_id):
    try:
        if trains_per_hour > 0:
            headway = 60 / trains_per_hour

            # Now creating a transit line directly in the network
            route = network.create_transit_line(line_id, vehicle_id, itinerary)
            route.description = description
            route.headway = headway

            #print "Transit line created: {} with headway {} minutes".format(line_id, headway)
            return route
        else:
            #print "Skipped line {} due to insufficient data for trains per hour.".format(line_id)
            return None
    except Exception as e:
        print "An error occurred with line {}: {} in scenario {}".format(line_id, e, str(scenario_id))
        return None
    

def update_route_segments(route, segment_attrib):
        """
        Updates the segments of a given transit line based on specified attributes.

        Parameters:
        route (object): The transit route object.
        segment_attrib (dict): A dictionary containing segment attributes such as dwell times,
                       speeds, and stop flags.
        """
        if route is not None:
            for i, segment in enumerate(route.segments(include_hidden=True)):
                if segment.link is None:
                    # Hidden segment at the end
                    segment.dwell_time = segment_attrib["dwell_times"][-1]
                else:
                    segment.transit_time_func = 1
                    segment.data1 = segment_attrib["speeds"][i]
                    segment.allow_alightings = segment_attrib["stop_flags"][i]
                    segment.allow_boardings = segment_attrib["stop_flags"][i]
                    segment.dwell_time = segment_attrib["dwell_times"][i]         

def temp_update_transit_vehicles(network, capacity_dict):
    new_vehicle_id_dict = {}
    existing_veh_desc = [str(veh.description) for veh in network.transit_vehicles()]
    existing_veh_id = sorted([veh.number for veh in network.transit_vehicles()])
    
    for vehicle_desc, veh_attrib in capacity_dict.iteritems():
        if veh_attrib["id"] in existing_veh_id or veh_attrib["old_name"].lower() in existing_veh_desc.lower():
            created = False
            
    return new_vehicle_id_dict

def change_vehicle_properties(vehicle, description="", auto_eq=0.0, seated_cap=0, total_cap=0):
        
        if description:
            vehicle.description = description
        
        if auto_eq != 0.0:
            vehicle.auto_equivalent = auto_eq
            
        if seated_cap != 0:
            vehicle.seated_capacity = seated_cap
        
        # Ensure total capacity is not less than seated capacity
        total_cap = max(total_cap, seated_cap)
        
        if total_cap != 0:
            vehicle.total_capacity = total_cap

def export_network(scenario_number, folder, time_peirod, attributes_to_export):
    file_name = "Modified {} Network.nwp".format(time_peirod)
    file_path = os.path.join(folder, file_name)

    try:
        # Re-load the tool each time to avoid stateful errors
        export_network_tool(scenario_number, file_path, attributes_to_export)
        print "Exported {}".format(file_name)
    except:
        print "Error encountered exporting %s network" % file_name
        
#load files
station_to_node_df = pd.read_csv(parameters["station_to_node_file"])
header_df = pd.read_csv(parameters["header_file"])
segment_df = pd.read_csv(parameters["segment_file"])
capacity_df = pd.read_csv(parameters["capacity_file"])


segment_df = segment_df.merge(station_to_node_df, left_on='from_station', right_on='station_name', how='left')
segment_df.rename(columns={'node_id': 'from_node_id'}, inplace=True)
segment_df.drop('station_name', axis=1, inplace=True) 

segment_df = segment_df.merge(station_to_node_df, left_on='to_station', right_on='station_name', how='left')
segment_df.rename(columns={'node_id': 'to_node_id'}, inplace=True)
segment_df.drop('station_name', axis=1, inplace=True) 

segment_df = pd.merge(segment_df, header_df[['line_id', 'union_node']], on='line_id', how='left')


segment_df['from_node_id'] = segment_df.apply(
    lambda row: row['union_node'] if row['from_station'] == 'Union' else row['from_node_id'], axis=1
)
segment_df['to_node_id'] = segment_df.apply(
    lambda row: row['union_node'] if row['to_station'] == 'Union' else row['to_node_id'], axis=1
)


segment_df = segment_df.dropna(subset=['union_node'])
segment_df.drop('union_node', axis=1, inplace=True)



# segment_df.to_csv("segment_test.csv", index=False)


all_station_stops = sorted(pd.concat([segment_df['from_node_id'], segment_df['to_node_id']]).astype(int).unique())


grouped_segments_by_line_dict = segment_df.groupby('line_id').apply(lambda x: x.to_dict('records')).to_dict()

header_df.set_index('line_id', inplace=True)
header_dict = {line: row.to_dict() for line, row in header_df.iterrows()}

capacity_df.set_index('name', inplace=True)
capacity_dict = {consist_type: row.to_dict() for consist_type, row in capacity_df.iterrows()}


# Import Networks if Flag is True
if parameters["IMPORT_NETWORK_FLAG"]:
    for network_file in os.listdir(parameters["network_folder"]):
        for scenario in parameters["source_scenarios"]:
            if scenario["time_period"] in network_file:
                print("Found source network: {} for scenario {}".format(network_file, scenario["scenario_id"]))
                import_network(parameters["network_folder"], network_file, scenario["scenario_id"])
                
# Load Scenarios and Process Network
source_scenario_list = []
for source_scenarios in parameters["source_scenarios"]:
    source_scenario = load_scenario(source_scenarios["scenario_id"])
    source_scenario_list.append(source_scenario)

for i, source_scenario in enumerate(source_scenario_list):
    source_network = source_scenario.get_network()
        
    new_vehicle_id_dict = {}
    existing_veh_desc = [str(veh.description.lower()) for veh in source_network.transit_vehicles()]
    existing_veh_id = sorted([veh.number for veh in source_network.transit_vehicles()])
    temp_vehs = []
    new_veh_nums = []
    
    for vehicle_desc, veh_attrib in capacity_dict.iteritems():
        if veh_attrib["id"] in existing_veh_id or veh_attrib["old_name"].lower() in existing_veh_desc:
            try:
                if veh_attrib["old_id"] == 0:
                    continue
                if veh_attrib["id"] > 900:
                    source_network.delete_transit_vehicle(veh_attrib["old_id"])
                    print "deleted vehicle - ", veh_attrib["id"]
                    continue
                
                new_veh_id = max(existing_veh_id) + 1
                new_vehicle = source_network.create_transit_vehicle(new_veh_id, veh_attrib["mode"])
                seated_cap = veh_attrib["seat"]
                total_cap = veh_attrib["capacity"]
                description = vehicle_desc
                change_vehicle_properties(new_vehicle,
                                       description=description,
                                       auto_eq=0.0,
                                       seated_cap=seated_cap,
                                       total_cap=total_cap)
                
                existing_veh_id.append(new_veh_id)
                
                              
                if veh_attrib["id"] < 900:
                    temp_vehs.append(new_vehicle)
                    new_veh_nums.append(int(veh_attrib["id"]))
                    new_vehicle_id_dict[vehicle_desc] = [temp_vehs, new_veh_nums]
                
                dependants = [line for line in source_network.transit_lines() if line.vehicle.number == veh_attrib["old_id"]]
                for line in dependants:
                    line.vehicle = new_vehicle
        
                
                
                if veh_attrib["old_id"] > 0:
                    source_network.delete_transit_vehicle(veh_attrib["old_id"])
            except Exception as e:
                print e
    
    source_scenario.publish_network(source_network)
    created_vehs = []
    source_network = source_scenario.get_network()
    
    
    for vehicle_desc, old_new_veh_map in new_vehicle_id_dict.iteritems():
        temp_vehs = old_new_veh_map[0]
        new_veh_ids = old_new_veh_map[1]
        new_veh_id_list = []
        for temp_veh, new_veh_id in zip(temp_vehs, new_veh_ids):  
            print temp_veh.description, temp_veh.id, new_veh_id
            try:
                if new_veh_id not in new_veh_id_list:
                    new_vehicle_replaced = source_network.create_transit_vehicle(new_veh_id, temp_veh.mode.id)
                    print "created vehicle for {} - {} ".format(new_vehicle_replaced, new_veh_id)
                    new_veh_id_list.append(new_veh_id)
                
                    for att in new_vehicle_replaced.network.attributes('TRANSIT_VEHICLE'):
                        new_vehicle_replaced[att] = temp_veh[att]
                        
                    dependants = [line for line in source_network.transit_lines() if line.vehicle.number == temp_veh.number]
                    for line in dependants:
                        line.vehicle = new_vehicle_replaced
                    created_vehs.append(new_vehicle_replaced.description)
                    source_network.delete_transit_vehicle(temp_veh.number)
            except Exception as e:
                print e 
        break
    source_scenario.publish_network(source_network)
    
    
    time_period = parameters["source_scenarios"][i]["time_period"]
    export_network(source_scenario.id, parameters["export_network_folder"], time_period, "ALL")
    
    print "done..."
                

Automatic pdb calling has been turned ON
starting....

Found source network: Assigned AM Peak Network.nwp for scenario 10
Imported network for scenario 10
Found source network: Assigned EV Evening Network.nwp for scenario 40
Imported network for scenario 40
Found source network: Assigned MD Midday Network.nwp for scenario 20
Imported network for scenario 20
Found source network: Assigned Midday OD S2S Network.nwp for scenario 20
Imported network for scenario 20
Found source network: Assigned ON Overnight Network.nwp for scenario 50
Imported network for scenario 50
Found source network: Assigned PM Peak Network.nwp for scenario 30
Imported network for scenario 30
Transit vehicle does not exist: 39
Transit vehicle does not exist: 41
Transit vehicle does not exist: 43
Transit vehicle does not exist: 42
Transit vehicle does not exist: 45
Transit vehicle does not exist: 47
Transit vehicle does not exist: 46
Transit vehicle does not exist: 38
Transit vehicle does not exist: 23
Transit vehicl

In [18]:
import pandas as pd

data =pd.read_csv("inputs/tts_extract_2.csv")

# # Data as provided (typically you would load this from a CSV file)
# data = {
#     "Line": ["BA10", "BA11", "BA12", "BA13", "BA14", "BA15", "BA16", "BA18", "BA20", "BA22", "BA24", "BA28", "BA30", "BA31", "BA32", "BA33", "BA34", "BA39", "BA41"],
#     "Unknown": [0] * 19,
#     "Female": [67, 48, 29, 34, 48, 47, 0, 0, 0, 43, 57, 0, 169, 34, 23, 23, 54, 23, 57],
#     "Male": [92, 47, 20, 0, 0, 54, 25, 25, 34, 0, 20, 47, 57, 103, 0, 0, 124, 34, 34],
#     "Total": [159, 95, 49, 34, 48, 101, 25, 25, 34, 43, 77, 47, 226, 137, 23, 23, 178, 57, 91]
# }

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# List of codes as per earlier discussion
my_list = ["HA", "T598", "T597", "T596", "T594","T593","GT01", "GT02", "GT03", "G9008", "GT05", 
           "GT06", "GT07", "GT09","GB", "TB", "T5", "M", "Y", "VZ", "D", "B", "W0", "K"]

# Function to check if any element of my_list is a substring of the line
def find_code(line):
    for code in my_list:
        if code in line and line.startswith(code,0):
            return code
    return "NA"

# Apply the function to create the new 'Code' column
df['Code'] = df['Line'].apply(find_code)
df.to_csv("proc_tts.csv")
# Print the resulting DataFrame
print(df)

     Time                         Line  Unknown  Female  Male  Total Code
0      AM                         BA10        0      67    92    159    B
1      AM                         BA11        0      48    47     95    B
2      AM                         BA12        0      29    20     49    B
3      AM                         BA13        0      34     0     34    B
4      AM                         BA14        0      48     0     48    B
5      AM                         BA15        0      47    54    101    B
6      AM                         BA16        0       0    25     25    B
7      AM                         BA18        0       0    25     25    B
8      AM                         BA20        0       0    34     34    B
9      AM                         BA22        0      43     0     43    B
10     AM                         BA24        0      57    20     77    B
11     AM                         BA28        0       0    47     47    B
12     AM                         BA30

In [16]:
text = "Owrld war"


text.startswith("w")


True